# Necessary imports

In [19]:
import pandas as pd
import numpy as np 
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import glob
import seaborn as sns
import re
import os
import io
from scipy.stats import boxcox

In [20]:
data = pd.read_csv('/Users/harshalsinha/Desktop/MSCS/Fall21/Stat581/FinalProject/US_Accidents_Dec20_updated.csv')
data

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-2716600,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.10891,-83.09286,40.11206,-83.03187,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2716601,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.86542,-84.06280,39.86501,-84.04873,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-2716602,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10266,-84.52468,39.10209,-84.52396,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-2716603,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10148,-84.52341,39.09841,-84.52241,0.219,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
4,A-2716604,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.06213,-81.53784,41.06217,-81.53547,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516059,A-4239402,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.00248,-117.37936,33.99888,-117.37094,0.543,At Market St - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day
1516060,A-4239403,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.76696,-117.14806,32.76555,-117.15363,0.338,At Camino Del Rio/Mission Center Rd - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day
1516061,A-4239404,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.77545,-117.84779,33.77740,-117.85727,0.561,At Glassell St/Grand Ave - Accident. in the ri...,...,False,False,False,False,False,False,Day,Day,Day,Day
1516062,A-4239405,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.99246,-118.40302,33.98311,-118.39565,0.772,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


In [21]:
data.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

Dropping the below mentioned features as data regarding them is collected only after the accident has occured. Hence, these cannot be used as predictors for serious accident prediction.

In [22]:
data = data.drop(['ID','Description','Distance(mi)', 'End_Time', 
              'End_Lat', 'End_Lng'], axis=1)

In [23]:
data.columns

Index(['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [24]:
features = ['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight']
print("Unique count of values in features:")
for feature in features:
    print(feature,data[feature].unique().size)

Unique count of values in features:
Severity 4
Start_Time 1037092
Start_Lat 590611
Start_Lng 603369
Number 37593
Street 93048
Side 2
City 10658
County 1671
State 49
Zipcode 177197
Country 1
Timezone 5
Airport_Code 1986
Weather_Timestamp 331749
Temperature(F) 776
Wind_Chill(F) 884
Humidity(%) 101
Pressure(in) 1008
Visibility(mi) 75
Wind_Direction 25
Wind_Speed(mph) 122
Precipitation(in) 188
Weather_Condition 117
Amenity 2
Bump 2
Crossing 2
Give_Way 2
Junction 2
No_Exit 2
Railway 2
Roundabout 2
Station 2
Stop 2
Traffic_Calming 2
Traffic_Signal 2
Turning_Loop 1
Sunrise_Sunset 3
Civil_Twilight 3
Nautical_Twilight 3
Astronomical_Twilight 3


The output above displays the count of unique values for each of the features. From the output we notice that the features - Country and Turning_Loop have only a single value. Moreover, as these two features are categorical features having only a single class these can be dropped as well.

In [7]:
data = data.drop(['Country','Turning_Loop'], axis=1)

# Cleaning up Categorical features:

We have the following categorical features:
'Side', 'Country', 'Timezone', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'.

We need to check the unique values in these features and clean up those values that are of the same class but have been spelled differently.

In [8]:
categorical_features = ['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Timezone',
       'Airport_Code', 'Wind_Direction', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal'
        , 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight']
for feature in categorical_features:
    print("Feature: ", feature, data[feature].unique())
    print()

Feature:  Severity [3 2 4 1]

Feature:  Start_Time ['2016-02-08 00:37:08' '2016-02-08 05:56:20' '2016-02-08 06:15:39' ...
 '2019-08-23 19:11:30' '2019-08-23 19:00:21' '2019-08-23 18:52:06']

Feature:  Start_Lat [40.10891  39.86542  39.10266  ... 34.120911 33.943599 34.239104]

Feature:  Start_Lng [ -83.09286   -84.0628    -84.52468  ... -118.416176 -117.14806
 -117.84779 ]

Feature:  Number [   nan  1887. 48999. ... 93671. 68630. 28237.]

Feature:  Street ['Outerbelt E' 'I-70 E' 'I-75 S' ... ' SE Dale Ln' ' San Marlo Way'
 '473-401 Cutoff Rd']

Feature:  Side ['R' 'L']

Feature:  City ['Dublin' 'Dayton' 'Cincinnati' ... 'Succasunna' 'Valleyford'
 'American Fork-Pleasant Grove']

Feature:  County ['Franklin' 'Montgomery' 'Hamilton' ... 'Mellette' 'Hodgeman' 'Ness']

Feature:  State ['OH' 'IN' 'KY' 'WV' 'MI' 'PA' 'CA' 'NV' 'MN' 'TX' 'MO' 'CO' 'OK' 'LA'
 'KS' 'WI' 'IA' 'MS' 'NE' 'ND' 'WY' 'SD' 'MT' 'NM' 'AR' 'IL' 'NJ' 'GA'
 'FL' 'NY' 'CT' 'RI' 'SC' 'NC' 'MD' 'MA' 'TN' 'VA' 'DE' 'DC' 'ME' 

From the above output we see that for the feature 'Wind_Direction' there are values that belong to the same class but have been spelled differently. We clean up such values.

In [11]:
data.loc[data['Wind_Direction']=='Calm','Wind_Direction'] = 'CALM'
data.loc[(data['Wind_Direction']=='West')|(data['Wind_Direction']=='WSW')|(data['Wind_Direction']=='WNW'),'Wind_Direction'] = 'W'
data.loc[(data['Wind_Direction']=='South')|(data['Wind_Direction']=='SSW')|(data['Wind_Direction']=='SSE'),'Wind_Direction'] = 'S'
data.loc[(data['Wind_Direction']=='North')|(data['Wind_Direction']=='NNW')|(data['Wind_Direction']=='NNE'),'Wind_Direction'] = 'N'
data.loc[(data['Wind_Direction']=='East')|(data['Wind_Direction']=='ESE')|(data['Wind_Direction']=='ENE'),'Wind_Direction'] = 'E'
data.loc[data['Wind_Direction']=='Variable','Wind_Direction'] = 'VAR'
print("Unique values for Wind_Direction after cleaning up: ", data['Wind_Direction'].unique())

Unique values for Wind_Direction after cleaning up:  ['SW' 'CALM' 'W' 'N' 'S' 'NW' 'E' 'SE' nan 'VAR' 'NE']


In [12]:
data['Weather_Condition'].unique()

array(['Light Rain', 'Overcast', 'Mostly Cloudy', 'Snow', 'Light Snow',
       'Cloudy', nan, 'Scattered Clouds', 'Clear', 'Partly Cloudy',
       'Light Freezing Drizzle', 'Light Drizzle', 'Haze', 'Rain',
       'Heavy Rain', 'Fair', 'Drizzle', 'Fog', 'Thunderstorms and Rain',
       'Patches of Fog', 'Light Thunderstorms and Rain', 'Mist',
       'Rain Showers', 'Light Rain Showers', 'Heavy Drizzle', 'Smoke',
       'Light Freezing Fog', 'Light Freezing Rain', 'Blowing Snow',
       'Heavy Thunderstorms and Rain', 'Heavy Snow', 'Snow Grains',
       'Squalls', 'Light Fog', 'Shallow Fog', 'Thunderstorm',
       'Light Ice Pellets', 'Thunder', 'Thunder in the Vicinity',
       'Fair / Windy', 'Light Rain with Thunder',
       'Heavy Thunderstorms and Snow', 'Light Snow Showers',
       'Cloudy / Windy', 'Ice Pellets', 'N/A Precipitation',
       'Light Thunderstorms and Snow', 'T-Storm', 'Rain / Windy',
       'Wintry Mix', 'Partly Cloudy / Windy', 'Heavy T-Storm', 'Sand',
       'Ligh

From the output above we see that there are a large number of values for the Weather_Condition feature. Upon reading through other research related to the analysis of car accidents in conjunction with weather conditions we came to the conclusion that the following weather conditions are of interest for our analysis as these conditions are more likely to cause an accident. These weather conditions of interest are: 'Clear', 'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow' and 'Fog'. 

In order to focus our analysis with regards to the correlation of car accidents and the corresponding weather conditions, we created separate features for the above mentioned weather conditions of interest.

Creating separate features(boolean) for: 'Clear', 'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow' and 'Fog'

In essence, for the rows where the value for the feature 'Weather_Condition' is equal to that particular weather condition of interest then it will be assigned the value true for that feature, else it is assigned the value false.

In [13]:
data['Clear'] = np.where(data['Weather_Condition'].str.contains('Clear', case=False, na = False), True, False)
data['Cloud'] = np.where(data['Weather_Condition'].str.contains('Cloud|Overcast', case=False, na = False), True, False)
data['Rain'] = np.where(data['Weather_Condition'].str.contains('Rain|storm', case=False, na = False), True, False)
data['Heavy_Rain'] = np.where(data['Weather_Condition'].str.contains('Heavy Rain|Rain Shower|Heavy T-Storm|Heavy Thunderstorms', case=False, na = False), True, False)
data['Snow'] = np.where(data['Weather_Condition'].str.contains('Snow|Sleet|Ice', case=False, na = False), True, False)
data['Heavy_Snow'] = np.where(data['Weather_Condition'].str.contains('Heavy Snow|Heavy Sleet|Heavy Ice Pellets|Snow Showers|Squalls', case=False, na = False), True, False)
data['Fog'] = np.where(data['Weather_Condition'].str.contains('Fog', case=False, na = False), True, False)

Now, that we have created features for the weather conditions of interest we will drop the original 'Weather_Condition' feature and for all the newly created features wherever the value of 'Weather_Condition' was null we assign the value NA. This ensures that for all the newly created features we have a boolean value or the value NA.

In [14]:
weather_conditions_of_interest = ['Clear','Cloud','Rain','Heavy_Rain','Snow','Heavy_Snow','Fog']
for weather_condition in weather_conditions_of_interest:
    data.loc[data['Weather_Condition'].isnull(),weather_condition] = data.loc[data['Weather_Condition'].isnull(),'Weather_Condition']
    data[weather_condition] = data[weather_condition].astype('bool')

data.loc[:,['Weather_Condition'] + weather_conditions_of_interest]

data = data.drop(['Weather_Condition'], axis=1)

# Calculate the percentage of missing values for the features

In [15]:
missing = pd.DataFrame(data.isnull().sum()).reset_index()
missing.columns = ['Feature', 'Missing_Percent(%)']
missing['Missing_Percent(%)'] = missing['Missing_Percent(%)'].apply(lambda x: x / data.shape[0] * 100)
missing.loc[missing['Missing_Percent(%)']>0,:]

,Feature,Missing_Percent(%)
4,Number,69.000715
7,City,0.005475
10,Zipcode,0.061673
11,Timezone,0.151841
12,Airport_Code,0.280199
13,Weather_Timestamp,1.996222
14,Temperature(F),2.838469
15,Wind_Chill(F),29.637007
16,Humidity(%),3.001786
17,Pressure(in),2.392643


The percentage of missing values for the feature 'Number' is more than 60%. Moreover, as the feature 'Number' corresponds to the street number of the accident site, it is not highly related to the severity of the accidents hence, we drop this feature.

In [16]:
data = data.drop(['Number'], axis=1)

From previous research we found out that the feature 'Precipitation(in)' can be a useful predictor hence, we create a separate feature 'Precipitation_NA' where we replace the missing values with the median.

In [17]:
data['Precipitation_NA'] = 0
data.loc[data['Precipitation(in)'].isnull(),'Precipitation_NA'] = 1
data['Precipitation(in)'] = data['Precipitation(in)'].fillna(data['Precipitation(in)'].median())

The count of missing values for the features: 'City','Zipcode','Airport_Code', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight','Astronomical_Twilight' is negligible as compared to the sample size. Hence, it makes sense to drop such rows that have missing values for these features.